In [7]:
import numpy as np
import pandas as pd
import os
from PIL import Image

In [3]:
path_to_read = 'interpolate_data_low_quality/'
files = []

for r, d, f in os.walk(path_to_read):
    for file in f:
        if '.png' in file:
            files.append(os.path.join(r, file))
files.sort()
np_files = np.asarray(files)

In [4]:
##Verification if each transient has 4 curves
#temp = 'SN000017'
#count = 0
#for f in np_files:
#    if (count == 4):
#        temp = f[29:37]
#        count = 0
#    if ((temp == f[29:37]) & (count < 4)):
#        count += 1
#    elif ((temp != f[29:37]) & (count < 4)):
#        print(f)

In [5]:
#set(types) -> {'IIL', 'IIP', 'II_', 'IIn', 'Ia_', 'Ib_', 'Ibc', 'Ic_'}
conversion = {'IIL' : 5, 'IIP' : 7, 'II_' : 4, 'IIn' : 6, 'Ia_' : 0, 'Ib_' : 1, 'Ibc' : 2, 'Ic_' : 3}

In [8]:
np_grouped = np_files.reshape(round(np_files.shape[0]/4), 4)
class_id = []
for g in np_grouped:
    class_id = class_id + [[g[0][29:37], g[0][38:41]]] 
types = []
for ids in class_id:
    types = types + [conversion[ids[1]]]
    
astr_objs = []
for i in range(len(np_grouped)):
    astr_objs = astr_objs + [class_id[i]]
    for file in np_grouped[i]:
        img = Image.open(file).convert('L')
        arr = np.array(img)
        astr_objs[-1] = astr_objs[-1] + [arr[5:35,10:56]]

print( "The quantity of objects is: ", len(astr_objs))

The quantity of objects is:  21317


In [12]:
labels = []
images = []
ids = []
for o in astr_objs:
    labels.append((conversion[o[1]])) #converting here to numbers to assure label in training
    images.append(o[2:])
    ids.append(o[0])
labels = np.asarray(labels)
images = np.asarray(images)
ids = np.asarray(ids)

train_n = 18000

train_labels = labels[0:train_n]
test_labels = labels[train_n:]

train_images = images[0:train_n]
test_images = images[train_n:]

train_ids = ids[0:train_n]
test_ids = ids[train_n:]


In [16]:
print('The shapes of the objects are:', ids.shape, labels.shape, images.shape, train_images.shape)

The shapes of the objects are: (21317,) (21317,) (21317, 4, 30, 46) (18000, 4, 30, 46)


In [17]:
train_images = np.around(train_images/255, decimals = 2)
test_images = np.around(test_images/255, decimals = 2)